Il seguente script crea le cartelle (dopo aver controllato che non esistano già) "audio", "docs" e "images". <br>
Dopodiché, itera la cartella principale spostando ogni file  seconda della propria estensione nella cartella di riferimento e compila un file csv in cui annota il nome, il formato ed il peso di tale file. 

In [42]:
# Script step 1

# importiamo i moduli che ci serviranno
import os, shutil, csv

# per prima cosa, creiamo le cartelle (se non esistono già) in cui verranno spostati i file.
for dirname in ["audio", "docs", "images"]:
    new_folder = os.path.join("FileOrganizer/files", dirname)
    if not os.path.exists(new_folder): 
        os.makedirs(new_folder)

# creiamo le tuple che contengono i formati file più comuni per ogni tipo di file che andremo ad analizzare:

docs_formats = (".txt", ".doc", ".docx", ".pages", ".odt", ".rtf", ".tex")
audio_formats = (".mp3", ".mp4", ".m4a", ".wma", ".flac", ",aac")
images_formats = (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff")


# controllo se il file di recape esiste ed eventualmente lo creo
if not os.path.exists("FileOrganizer/files/recap.csv"): # se non esiste
    recap = open("FileOrganizer/files/recap.csv", "w", newline='') # lo creo
    writer = csv.writer(recap)
    writer.writerow(["name", "type", "size (B)"]) # scrivo header del csv
    
else: # se esiste già
    recap = open("FileOrganizer/files/recap.csv", "a", newline='') # lo apro in 'append' con 'a'
    writer = csv.writer(recap)

# iteriamo in ordine alfabetico i file nella cartella "files":
for file in sorted(os.listdir("FileOrganizer/files")):

    name, ext = os.path.splitext(file) # separo il nome del file dall'estensione
    percorso = "FileOrganizer/files/" + file # specifica il percorso del file
    size = os.stat(percorso).st_size # calcola la dimensione del file

    #usiamo il metodo "endswith" per prendere in considerazione i formati file e smistarli nelle relative cartelle:
    if ext in docs_formats:
        dst = ("FileOrganizer/files/docs/" + file) # dst sarà la cartella in cui verrrà spostato il file
        shutil.move(percorso, dst) # hutil.move serve a spostare il file dalla cartella originale a quella richiesta
        writer.writerow([name, "docs", size]) # aggiorno recap
        print(f'{name} type:docs size:{size}B')
        
    elif ext in audio_formats:
        dst = ("FileOrganizer/files/audio/" + file)
        shutil.move(percorso, dst)
        writer.writerow([name, "audio", size])   
        print(f'{name} type:audio size:{size}B')
        
    elif ext in images_formats:
        dst = ("FileOrganizer/files/images/" + file)
        shutil.move(percorso, dst)
        writer.writerow([name, "image", size])
        print(f'{name} type:image size:{size}B')
               
    else:
        if name not in ["audio", "docs", "images", "recap"]: #per ignorare le cartelle e il file di recap
            print("Formato non riconosciuto.", name)
        
        
recap.close() #chiudo il file di recap alla fine della scrittura     

bw type:image size:94926B
ciao type:docs size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:docs size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


Il seguente script, invece, analizza le immagini e ne estrae le informazioni di colore sfruttando la libreria Numpy.<br>
Infine stampa una tabella contenente le informazioni raccolte.

In [114]:
# importiamo i moduli che ci serviranno
import os
import numpy as np
from PIL import Image
from tabulate import tabulate

# creiamo una tabella vuota che verrà riempita ad ogni iterazione
tab = []

# iteriamo i file nella cartella "images"
for i in os.listdir("FileOrganizer\\files\\images"):
    file = ("FileOrganizer\\files\\\images\\" + i) #specifica il percorso del file
    im = Image.open(file) #apre l'immagine
    im_array = np.asarray(im) #trasforma l'immagine in un array numpy
    name = i.split(".")[0] #separa il nome dal formato
    height = np.shape(im_array)[0] #restituisce l'altezza dell'immagine
    width = np.shape(im_array)[1] #restituisce la larghezza dell'immagine

# usiamo "ndim" per effettuare diverse operazioni a seconda del numero delle dimensioni dell'array
    if im_array.ndim == 2: # se le dimensioni sono 2, vuol dire che si tratta di un'immagine in bianco e nero
        grayscale = np.mean(im_array) # calcoliamo la scala del grigio
        R, G, B, ALPHA = 0.00, 0.00, 0.00, 0.00
        
    elif im_array.ndim == 3: # se le dimensioni sono 3, vuol dire che si tratta di un'immagine a colori
        grayscale = 0.00 # essendo un'immagine a colori, la scala di grigio sarà 0
        R = np.mean(im_array[:, :, 0]) # con l'uso degli slicing degli array calcoliamo la scala di colori facendone la media
        G = np.mean(im_array[:, :, 1])
        B = np.mean(im_array[:, :, 2])
        
        if np.shape(im_array)[2] == 3: # se gli elementi della terza dimensione sono 3, l'immagine è RGB, quindi ALPHA sarà 0
            ALPHA = 0.00
            
        elif np.shape(im_array)[2] == 4:# se gli elementi della terza dimensione sono 4, l'immagine è RGBA, quindi andremo a calcolare il valore di ALPHA
            ALPHA = np.mean(im_array[:, :, 3])
            
# creiamo una lista con le informazioni ottenute e l'aggiungiamo alla lista "tabella", creando così una lista di liste
    tab.append([name, height, width, grayscale, R, G, B, ALPHA])
    
# infine con il modulo "tabulate" creiamo e stampiamo la tabella, specificande nell'headers le etichette delle colonne 
headers = ["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"]
print(tabulate(tab, headers=headers,
               tablefmt="fancy_grid",
               floatfmt='.2f'))

╒══════════╤══════════╤═════════╤═════════════╤══════════╤══════════╤══════════╤═════════╕
│ name     │   height │   width │   grayscale │        R │        G │        B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪══════════╪══════════╪══════════╪═════════╡
│ bw       │      512 │     512 │     21.4801 │   0      │   0      │  0       │   0     │
├──────────┼──────────┼─────────┼─────────────┼──────────┼──────────┼──────────┼─────────┤
│ daffodil │      500 │     335 │      0      │ 109.251  │  85.5552 │  4.96729 │   0     │
├──────────┼──────────┼─────────┼─────────────┼──────────┼──────────┼──────────┼─────────┤
│ eclipse  │      256 │     256 │      0      │ 109.05   │ 109.522  │ 39.849   │ 133.591 │
├──────────┼──────────┼─────────┼─────────────┼──────────┼──────────┼──────────┼─────────┤
│ trump    │      183 │     275 │      0      │  97.0108 │  98.9941 │ 90.9238  │   0     │
╘══════════╧══════════╧═════════╧═════════════╧══════════╧══════════╧══════════╧═════════╛